In [1]:
import json
import urllib.request
import pymongo
from bs4 import BeautifulSoup
import pandas as pd
import time


def save_current_file(file_text, filetype='json'):
    current_page_out = open('output.{}'.format(filetype), 'w+')
    current_page_out.write(file_text)
    current_page_out.close()


def scrape_review_page(movie_id):
    movie_review_url = 'https://www.imdb.com/title/tt{}/reviews/'.format(str(movie_id).zfill(7))
    current_page = urllib.request.urlopen(movie_review_url)
    soup = BeautifulSoup(current_page, 'html.parser')

    movie_title = soup.select('meta [property="og:title"]')[0].get('content')
    current_movie = {
        "movie_id": movie_id,
        "title": movie_title,
        "ratings": []
    }
    for tag in soup.find_all(attrs={'class': "imdb-user-review"}):
        try:
            user_id = tag.select('.display-name-link a')[0].get('href')[8:15]
            user_rating = tag.select('.rating-other-user-rating span')[0].text

            current_movie['ratings'].append({
                "user_id": user_id,
                "user_rating": user_rating
            })
        except Exception as e:
            print("Error caused by movie_id : {}. Exception: {}".format(movie_id, e))
    return current_movie


def scrape_index_page(movie_id):
    movie_index_url = 'https://www.imdb.com/title/tt{}/'.format(str(movie_id).zfill(7))
    current_page = urllib.request.urlopen(movie_index_url)
    index_soup = BeautifulSoup(current_page, 'html.parser')
    current_page_json = index_soup.find('script', attrs={'type': 'application/ld+json'}).text
    return current_page_json


def make_connection(collection_name):
    mongodb_server = "localhost"
    mongodb_port = 27017
    mongodb_db = "imdb"
    mongodb_collection = collection_name
    connection = pymongo.MongoClient(mongodb_server, mongodb_port)
    db = connection[mongodb_db]
    collection = db[mongodb_collection]
    return collection


def save_current_instance(collection, current_page_json):
    collection.insert_one(current_page_json)


def get_movie_ids(num=500):
    links_data = pd.read_csv('./data/links.csv')
    movie_ids = list(links_data.imdbId)
    return movie_ids[:num]


def scrape_and_store_mongodb():
    print("Starting Scraping")
    print("=" * 20)
    start = time.time()
    num_movies = 1000
    current_index = 0
    movie_ids = get_movie_ids(num=num_movies)
    movies_collection = make_connection(collection_name='movies')
    reviews_collection = make_connection(collection_name='reviews')
    for movie_id in movie_ids:
        movies_collection.insert_one(json.loads(scrape_index_page(movie_id)))
        reviews_collection.insert_one(scrape_review_page(movie_id))
        current_index = current_index + 1
        print("Movies completed: {}/{}".format(current_index, num_movies))
    print("Time taken: {}".format(time.time() - start))
    print("=" * 20)


# scrape_and_store_mongodb()

collection = make_connection(collection_name='reviews')

data = pd.DataFrame(list(collection.find()))

In [2]:
data.head()

,_id,movie_id,title,user_id,user_rating
0,5c6094e95ecae14da2cc3194,114709,Toy Story (1995),3922673,9
1,5c6094e95ecae14da2cc3195,114709,Toy Story (1995),4445210,10
2,5c6094e95ecae14da2cc3196,114709,Toy Story (1995),1173088,10
3,5c6094e95ecae14da2cc3197,114709,Toy Story (1995),16117882,10
4,5c6094e95ecae14da2cc3198,114709,Toy Story (1995),2446936,9


In [12]:
raw_data = data.drop(labels=['_id', 'title'], axis=1)

In [22]:
raw_data['user_rating'] = pd.to_numeric(raw_data['user_rating'])
raw_data.head()

,movie_id,user_id,user_rating
0,114709,3922673,9
1,114709,4445210,10
2,114709,1173088,10
3,114709,16117882,10
4,114709,2446936,9


In [23]:
raw_data_pivot = raw_data.pivot_table(values='user_rating', index='movie_id')

In [24]:
raw_data_pivot.corr() 

,user_rating
user_rating,1.0
